## Case #001: The Phantom Peddler
#### A newly launched, high-margin product is posting massive sales, but most of the commission is going to a single salesperson who nobody in the office has ever seen.
Objectives:
* Identify the ProductID for the new product (e.g., 'Touring-1000 Blue, 46').
* Isolate the single SalesPersonID responsible for the most of those sales.
* Cross-reference the SalesPersonID with the employee roster to reveal the "phantom's" name and job title.

In [ ]:
-- Looking for the ProductID for the 'Touring-1000 Blue, 46'
select P.ProductID
from Production.Product AS P
where P.Name = 'Touring-1000 Blue, 46'

-- Looking for all orders made for new product
select sod.SalesOrderID
from Sales.SalesOrderDetail sod 
where sod.ProductID = '966'

-- Looking for the pesron who made most of the sales of new product
select top (1) soh.SalesPersonID, sum(sod.OrderQty)
from Sales.SalesOrderHeader soh 
join Sales.SalesOrderDetail sod 
on soh.SalesOrderID = sod.SalesOrderID
where sod.ProductID = '966'
group by soh.SalesPersonID
having soh.SalesPersonID is not null
order by sum(sod.OrderQty) desc

-- After we found ID of this person searching for his First and Last name
select p.FirstName, p.LastName 
from Person.Person p 
where p.BusinessEntityID = (select top (1) soh.SalesPersonID
from Sales.SalesOrderHeader soh 
join Sales.SalesOrderDetail sod 
on soh.SalesOrderID = sod.SalesOrderID
where sod.ProductID = '966'
group by soh.SalesPersonID
having soh.SalesPersonID is not null
order by sum(sod.OrderQty) desc)


## Case #002: The Bleeding Margin
#### The 'Mountain-200' line is moving units, but the profit margin is bleeding out. Someone is giving a sweetheart deal to a specific customer for a fat kickback.
Objectives:
* Find all sales of 'Mountain-200' products where the UnitPriceDiscount is abnormally high (e.g., > 30%).
* Identify the SalesPersonID and CustomerID that appear most frequently on these shady deals.
* Unmask the salesperson and the customer's store name.


In [ ]:
-- Looking for the ProductID for the 'Mountain-200'
select p.ProductID
from Production.Product AS p
where p.Name LIKE N'%Mountain-200%'

-- Looking for all orders made for this product that got a discount more than 30%
select sod.SalesOrderID, sod.UnitPriceDiscount
from Sales.SalesOrderDetail sod 
where sod.ProductID in (select p.ProductID
from Production.Product AS p
where p.Name LIKE N'%Mountain-200%')
and sod.UnitPriceDiscount > 0.3

-- Looking for the sales pesron who made such sale and customer ID
select soh.SalesPersonID, soh.CustomerID
from Sales.SalesOrderHeader soh 
where soh.SalesOrderID in (select sod.SalesOrderID
from Sales.SalesOrderDetail sod 
where sod.ProductID in (select p.ProductID
from Production.Product AS p
where p.Name LIKE N'%Mountain-200%')
and sod.UnitPriceDiscount > 0.3)

-- After we found ID of this sales person searching for his First and Last name
select p.FirstName, p.LastName 
from Person.Person p 
where p.BusinessEntityID = (select soh.SalesPersonID
from Sales.SalesOrderHeader soh 
where soh.SalesOrderID in (select sod.SalesOrderID
from Sales.SalesOrderDetail sod 
where sod.ProductID in (select p.ProductID
from Production.Product AS p
where p.Name LIKE N'%Mountain-200%')
and sod.UnitPriceDiscount > 0.3))

-- After we found ID of this customer searching for store name
select s.Name
from Sales.Store s   
join Sales.Customer c 
on c.StoreID = s.BusinessEntityID
where c.CustomerID = (select soh.CustomerID
from Sales.SalesOrderHeader soh 
where soh.SalesOrderID in (select sod.SalesOrderID
from Sales.SalesOrderDetail sod 
where sod.ProductID in (select p.ProductID
from Production.Product AS p
where p.Name LIKE N'%Mountain-200%')
and sod.UnitPriceDiscount > 0.3))

## Case #003: The Case of the Empty Storefront
#### A store, "Next-Door Bike Shop," has zero sales in the last year, but our records show it's still listed as one of our "Active" customers.
Objectives:
* Find the CustomerID for the store named "Next-Door Bike Shop".
* Query the SalesOrderHeader table to confirm they have zero orders in the last 12 months.
* Check the Customer table to see who the SalesPersonID on this "ghost" account is.

In [ ]:
-- Find IDs of the store
select c.CustomerID
from Sales.Store s   
join Sales.Customer c
on c.StoreID = s.BusinessEntityID
where s.Name = 'Global Plaza'

-- Make sure there was no sales for the last 12 month
select *
from Sales.SalesOrderHeader soh 
where soh.CustomerID in (select c.CustomerID
from Sales.Store s   
join Sales.Customer c
on c.StoreID = s.BusinessEntityID
where s.Name = 'Global Plaza')
and soh.OrderDate > DATEADD(MONTH, -12, SYSUTCDATETIME());

-- Find the sales person First and Last name
select p.FirstName, p.LastName 
from Sales.Store s   
join Person.Person p
on p.BusinessEntityID = s.SalesPersonID 
where s.Name = 'Global Plaza'

## Case #004: The Discount King
#### One salesperson's average discount rate is 10x the company average, and all his high-discount sales are going to a single territory.
Objectives:
* Calculate the average UnitPriceDiscount for all SalesPersonIDs.
* Identify the one salesperson with a massively outlying average.
* List all the territories this salesperson sold to and find the one receiving all the discounts.

In [ ]:
-- Calculate the average discount and find the sales person that gives the biggest discount
select top (1) avg(sod.UnitPriceDiscount) as average , soh.SalesPersonID
from Sales.SalesOrderDetail sod 
join Sales.SalesOrderHeader soh 
on sod.SalesOrderID = soh.SalesOrderID 
group by soh.SalesPersonID 
order by average desc

-- Find the territory fro this sales person
select soh.TerritoryID 
from Sales.SalesOrderHeader soh 
join Sales.SalesTerritory st 
on soh.TerritoryID = st.TerritoryID
where soh.SalesPersonID = 285

## Case #005: The Sunday Spender
#### A single customer places all their (massive) orders on Sunday, when no sales staff are working, and they all get approved.
Objectives:
* Find all orders placed on a Sunday (using OrderDate).
* Identify the CustomerID that dominates this list.
* Check if these orders have a SalesPersonID attached (they shouldn't) and see if their CreditCardID is valid.

In [ ]:
-- Find customer with hihgest ammount of purchases on sunday
select top 1 count(soh.CustomerID), soh.CustomerID
from Sales.SalesOrderHeader as soh
where DATENAME(WEEKDAY, OrderDate) = 'Sunday'
group by soh.CustomerID
order by count(soh.CustomerID) desc

-- Check is there an attached sales person for these orders
select soh.SalesPersonID 
from Sales.SalesOrderHeader as soh
where soh.CustomerID = (select top 1 soh.CustomerID
from Sales.SalesOrderHeader as soh
where DATENAME(WEEKDAY, OrderDate) = 'Sunday'
group by soh.CustomerID
order by count(soh.CustomerID) desc)

-- Check is credit card id valid
select soh.CreditCardID 
from Sales.SalesOrderHeader as soh
where soh.CustomerID = (select top 1 soh.CustomerID
from Sales.SalesOrderHeader as soh
where DATENAME(WEEKDAY, OrderDate) = 'Sunday'
group by soh.CustomerID
order by count(soh.CustomerID) desc)

## 

## Case #006: The Overpaid Executive Background: Something smells off in payroll.
#### A few department managers seem to earn way above their peers — and even more than their own department’s average salary. HR wants to know who’s cashing in while everyone else scrapes by.
Objectives:
* Use HumanResources.EmployeePayHistory and HumanResources.EmployeeDepartmentHistory. For each department, calculate the average rate. 
* Find employees whose current pay is greater than 2× the department average. 
* List their BusinessEntityID, DepartmentID, Rate, and the calculated average department rate.

In [ ]:
-- Find average rate for the department
select avg(rate)
from HumanResources.EmployeePayHistory eph 
join HumanResources.EmployeeDepartmentHistory edh 
on eph.BusinessEntityID = edh.BusinessEntityID  
join HumanResources.Department d 
on d.DepartmentID = edh.DepartmentID
group by d.DepartmentID 

-- Function to find average rate for the department by department ID
create function dbo.GetDepartmentAverageRate (@DepartmentID INT)
returns decimal(10,2)
as
begin
declare @AvgRate decimal(10,2);

select @AvgRate = AVG(eph.Rate)
from HumanResources.EmployeePayHistory as eph
join HumanResources.EmployeeDepartmentHistory as edh
on eph.BusinessEntityID = edh.BusinessEntityID
where edh.DepartmentID = @DepartmentID
and edh.enddate is null; 
return @AvgRate;
end;

-- Find employees with salary 2 times higher than avarage for their department
select eph.BusinessEntityID, edh.DepartmentID, eph.Rate, dbo.GetDepartmentAverageRate(edh.DepartmentID) as AvgDeptRate
from HumanResources.EmployeePayHistory eph
join HumanResources.EmployeeDepartmentHistory edh
on eph.BusinessEntityID = edh.BusinessEntityID
where eph.Rate > dbo.GetDepartmentAverageRate(edh.DepartmentID) * 2;

## #007: The Online Ghosts
#### Some sales just don’t make sense. Orders are flowing in, money’s being counted —
but no salesperson’s name is on the receipt. Someone’s selling in the shadows… or maybe the machines are doing it themselves.
Objectives:
* Find all sales territories that have orders without an assigned salesperson (SalesPersonID IS NULL).
* Show some examples of these “ghost” orders — when and where they happened.

In [ ]:
select 
    coalesce(st.Name, 'No Territory') as TerritoryName,
    count(*) as OrdersWithoutSalesperson
from Sales.SalesOrderHeader as soh
left join Sales.SalesTerritory as st
    on soh.TerritoryID = st.TerritoryID
where soh.SalesPersonID is null
group by st.Name
order by OrdersWithoutSalesperson desc;

select top 50
    soh.SalesOrderID,
    soh.OrderDate,
    soh.CustomerID,
    soh.TerritoryID,
    soh.OnlineOrderFlag
from Sales.SalesOrderHeader as soh
where soh.SalesPersonID is null
order by soh.OrderDate desc;